In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys
from pathlib import Path

sys.path.append("../")

import numpy as np
import pandas as pd

In [ ]:
from choice_learn.datasets import load_modecanada

transport_df = load_modecanada(as_frame=True)

# Following torch-choice guide:
transport_df = transport_df.loc[transport_df.noalt == 4]

items = ["air", "bus", "car", "train"]
transport_df.income = transport_df.income.astype("float32")

In [ ]:
# Initialization of the ChoiceDataset
from choice_learn.data import ChoiceDataset
dataset = ChoiceDataset.from_single_df(df=transport_df,
                                       contexts_features_columns=["income"],
                                       contexts_items_features_columns=["cost", "freq", "ovt", "ivt"],
                                       items_id_column="alt",
                                       contexts_id_column="case",
                                       choices_column="choice",
                                       choice_mode="one_zero")

In [ ]:
from choice_learn.models import ConditionalMNL
from choice_learn.models.base_model import BaseMixtureModel

In [ ]:
hpp = {
    "parameters": {"income": "item",
 "cost": "constant",
 "freq": "constant",
 "ovt": "constant",
 "ivt": "item-full",
 "intercept": "item"},
 "optimizer": "lbfgs",
 "epochs": 10,
}
mixMNL = BaseMixtureModel(model_class=ConditionalMNL, 
                          model_parameters=hpp,
                          latent_classes=4, epochs=10, fit_method="fit")

In [ ]:
mixMNL.instantiate()
mixMNL._em_fit(dataset)